## Import Libiray 

In [2]:
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt


## Define Parameters 

In [3]:
product_list = ['AUDUSD','AUDCHF','AUDCAD','NZDCHF','USDCHF','NZDCAD','EURCHF','AUDNZD','USDCAD','GBPNZD']

para_num = 4
ts_data_save = 'TS_data/'
model_save = 'cnn_model_save/'


# Hyper parameters
EPOCH = 10000
learning_rate = 1e-2

# Network Parameters
CNN_input_dim = 1
CNN_ouput_dim = 4

train_ratio = 0.8

## DataLoader

In [4]:
ts_d = pd.read_csv(ts_data_save+'AUDUSD1440_Data.csv')
data = np.zeros((len(ts_d),CNN_input_dim, len(product_list),CNN_ouput_dim ), dtype=float)
for k in range(len(product_list)):
    product = product_list[k]

    ts_d = pd.read_csv(ts_data_save+product+'1440_Data.csv')
    ts_d = ts_d.dropna(axis=0,how='any')
    ts_d = ts_d.drop(['Date','Year','Month','Day','Hour','Minute','Volume','RSI','MACD_M','MACD_S','STO_K','STO_D'],axis=1)
    ts_d = ts_d.iloc[:,:].values
    
    for i in range(len(ts_d)):
        temp = np.zeros((len(product_list),para_num))
        #temp = np.zeros((para_num))
        temp = ts_d[i]
        data[i][0][k] = temp

# The save name is the first product
product = product_list[0]

In [5]:

ts_d_len = len(ts_d)
train_num = math.ceil(train_ratio*ts_d_len)
test_num = math.ceil(train_ratio*ts_d_len)
# Use index to assign value to Train data and Test data
train_data = data[0:train_num]
test_data = data[train_num: ts_d_len]
# Load the data into training set
train_X = np.zeros((train_num,CNN_input_dim, len(product_list),CNN_ouput_dim), dtype=float)
train_Y = np.zeros((train_num, CNN_ouput_dim), dtype=float)

for i in range(train_num):
    train_X[i] = train_data[i]
    
for i in range(train_num):
    train_Y[i] = train_data[i][0][0]

# Load the data into testing set
test_X = np.zeros((test_num, CNN_input_dim, len(product_list),CNN_ouput_dim), dtype=float)
test_Y = np.zeros((test_num, CNN_ouput_dim), dtype=float)
for i in range(test_num):
    test_X[i] = train_data[i]
    
for i in range(test_num):
    test_Y[i] = train_data[i][0][0]

# Encapsulate the data into Tensor
train_X = torch.tensor(train_X,dtype=torch.float32).cuda()
train_Y = torch.tensor(train_Y,dtype=torch.float32).cuda()
test_X = torch.tensor(test_X,dtype=torch.float32).cuda()
test_Y = torch.tensor(test_Y,dtype=torch.float32).cuda()
print(train_X.shape)
print('Finish Loading........')
print('training data:',train_num,'test_data:',test_num)

torch.Size([1600, 1, 10, 4])
Finish Loading........
training data: 1600 test_data: 1600


## Define CNN Network

In [6]:
class CNN(nn.Module):
    def __init__(self,input_size=CNN_input_dim, output_size=CNN_ouput_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            # IN(1*10*4)
            nn.Conv2d(
                in_channels = input_size,
                out_channels = 16,
                kernel_size = 2,
                stride = 1,
            ),
            # OUT(16*10*4)
            # IN (16*10*4
            nn.ReLU(),
            # OUT(16*10*4
            # IN(16*10*4
            nn.AvgPool2d(kernel_size=2)
            # OUT(16*5*2
        )
        self.conv2 = nn.Sequential(
            # IN(16*5*2
            nn.Conv2d(
                in_channels = 16,
                out_channels = 32,
                kernel_size = 2,
                stride = 1,
                padding =2,
            ),
            # OUT(32*5*2
            # IN(32*5*2
            nn.ReLU(),
            # OUT(32*5*2)
            # IN(32*5*2
            nn.AvgPool2d(2),
            # OUT(32*3*2)
        )
        self.out = nn.Linear(32*3*2, output_size)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [7]:
cnn = CNN().cuda()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (out): Linear(in_features=192, out_features=4, bias=True)
)


## Define optimizer and loss func

In [8]:
optimizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate)
loss_function = nn.MSELoss()
#loss_function = nn.CrossEntropyLoss()

## Define training function

In [9]:

def train_model(train_X, save_model_name):
    losses = []
    print("Training Start. Epochs=", EPOCH)
    for i in range(EPOCH):
        output = cnn(train_X)
        loss = loss_function(output, train_Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if (i+1) % 50 == 0:
#             test_output = cnn(test_X)
#             pred_y = torch.max(test_output,1)[1].cuda().data.squeeze()
#             accuracy = sum(pred_y == test_Y) / test_y.size(0)
#             print('Epoch', epoch, '|test accuracy|:%.4f'% accuracy)
             print('Epoch:',i+1,'Loss:', loss.item())
    
    torch.save(cnn, model_save+save_model_name)
    return loss.item(), losses

def test_model(model_name, test_X):
    # Load the trained model
    load_cnn = torch.load(model_save+model_name)
    # Forward the Test Data
    test_output = load_cnn.forward(test_X)
    # Calculate the loss
    loss = loss_function(test_output, test_Y)

    return test_output, loss.item()

In [ ]:
if __name__ == '__main__':
    model_name=product+'.pkl'
    train_loss, losses = train_model(train_X, model_name)
    

In [10]:
test_output, test_loss = test_model(product+'.pkl', test_X)
print("Train loss = ", train_loss," Test loss = ", test_loss)

Train loss =  5.324787707650103e-06  Test loss =  5.510259597940603e-06


In [ ]:
test_output, _ = test_model(model_name, test_X)
test_output_plot = test_output.cpu().detach().numpy()
test_y_plot = test_Y.cpu().detach().numpy()

In [14]:
out = pd.DataFrame(test_output_plot)
out.to_csv('CNN_Pred/'+product+'.csv',index=False)